In [10]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import csv
import os
import pickle
from pathlib import Path
from tqdm import tqdm
from PIL import ImageOps, Image

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

In [82]:
EXERCISE = 'half-burpee'
FROM_CLASS = 'in'
seq_len = 16
seq_shape = [seq_len, 99]

## Old way
Seq level

In [30]:
# target_seq = 3106
# target_vid = 216

In [31]:
# image_dirs = ['/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up']
# accepted_exts = ('.jpg', '.jpeg','.png')
# all_file_paths = []
# for image_dir in image_dirs:
#     for path, subdir, files in os.walk(image_dir):
#         # for file in sorted(files, key=lambda x: int(x.split('_')[0])):
#         for file in files:
#             if file.lower().endswith(accepted_exts):
#                 full_path = os.path.join(path, file)
#                 print(full_path)
#                 all_file_paths.append(full_path)

In [6]:
# import random

In [7]:
# random.shuffle(all_file_paths)

In [8]:
# len(all_file_paths)

## Extract keypoints from path

In [9]:
# SEQ_LEN = 8
# data = []
# names = []
# error_files = []
# for image_path in tqdm(all_file_paths):
#     image = cv2.imread(image_path)
#     name_with_ext = image_path.split(os.path.sep)[-1]
#     name = name_with_ext[:name_with_ext.rfind('.')]
#     with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
#         with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose2:
#             results = pose.process(image)

#             try:
#                 pose_coor = results.pose_landmarks.landmark
#                 # no need z here
#                 body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
#                 stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
#                 data.append(stacked_kps)
#                 names.append(name)

#             except:
#                 error_files.append(image_path)
                
#             # augment flip horizontal
#             flipped_image = np.flip(image, axis=1)   
#             aug_results = pose2.process(flipped_image)
            
#             try:
#                 pose_coor = aug_results.pose_landmarks.landmark
#                 # no need z here
#                 body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
#                 stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
#                 data.append(stacked_kps)
#                 names.append(name+'_aug')

#             except:
#                 error_files.append(image_path+'_aug')

In [10]:
# error_files

In [11]:
# data = np.array(data)
# data.shape

In [12]:
# og_len = len(names)
# og_len

In [13]:
# target_seq / og_len

In [14]:
# # stack data more
# STACK_MORE = 4
# stacked_names = names*STACK_MORE
# stacked_data = np.tile(data, (STACK_MORE, 1, 1))
# stacked_data.shape

In [15]:
# stacked_names

In [16]:
# stacked_data = stacked_data.reshape((-1, 8, 99))
# stacked_data

In [17]:
# stacked_data.shape

In [18]:
# np.min(stacked_data), np.mean(stacked_data),np.max(stacked_data)

# New way

In [83]:
# get all paths and count
main_dir = Path(f'/home/u6188041/jom/Dataset/EzFit_dataset/keypoints_seq/len_{seq_len}/{EXERCISE}/{EXERCISE}-{FROM_CLASS}')
accepted_exts = ('.npy')
all_file_paths = []
for path, subdir, files in os.walk(main_dir):
    for file in files:
        if file.lower().endswith(accepted_exts):
            full_path = os.path.join(path, file)
            all_file_paths.append(full_path)

In [84]:
amt = len(all_file_paths)
# amt = stacked_data.shape[0]
amt

625

In [85]:
np.random.seed(42)
noise = np.random.normal(scale=0.01, size=[amt]+seq_shape)
noise.shape

(625, 16, 99)

In [86]:
noise.min(), noise.mean(), noise.max()

(-0.04829436008511157, -1.5499738110986524e-05, 0.04678949100376724)

In [44]:
## OLD
# noisy_stacked_data = stacked_data + noise

In [24]:
# noisy_stacked_data.min(), noisy_stacked_data.mean(),noisy_stacked_data.max()

In [25]:
# np.save('saved_data/new_pushup_idle_class_len_8', noisy_stacked_data)

In [26]:
# test
# subfiles = sorted(os.listdir(main_dir/'28_3'), key=lambda x: int(x[:x.rfind('.')].split('_')[-1]))
# subfiles

In [87]:
# Split Video based
running_number = 0
base_outdir = Path(f'/home/u6188041/jom/Dataset/EzFit_dataset/keypoints_seq/len_{seq_len}/{EXERCISE}/idle')
for vid_name in tqdm(os.listdir(main_dir)):
    # format of file .npy will be like 1_2_1.npy
    subfiles = sorted(os.listdir(main_dir/vid_name), key=lambda x: int(x[:x.rfind('.')].split('_')[-1]))
    number = len(subfiles)
    try:
        first = subfiles[0]
    except:
        print('[IGNORED] Empty dir:', vid_name)
        continue
    arr_first = np.load(main_dir/vid_name/first)[0]
    arr_first = np.expand_dims(arr_first, axis=0)
    arr_first = np.tile(arr_first, (seq_len, 1))
    stacked_arr = np.tile(arr_first, (number, 1, 1))
    assert stacked_arr.shape == tuple([number]+seq_shape), 'Invalid stacked shape'
    save_dir = base_outdir / vid_name
    save_dir.mkdir(parents=True, exist_ok=True)
    for seq_num in range(number):
        noisy_arr = stacked_arr[seq_num,:] + noise[running_number,:]
        np.save(save_dir / (vid_name+'_'+str(seq_num)), noisy_arr)
        running_number += 1

100%|███████████████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 1152.08it/s]

[IGNORED] Empty dir: 19_2
[IGNORED] Empty dir: 98_2
[IGNORED] Empty dir: 38_2
[IGNORED] Empty dir: 100_4
[IGNORED] Empty dir: 21_2
[IGNORED] Empty dir: 74_2
[IGNORED] Empty dir: 17_2
[IGNORED] Empty dir: 10_2
[IGNORED] Empty dir: 4_2
[IGNORED] Empty dir: 26_2
[IGNORED] Empty dir: 89_2
[IGNORED] Empty dir: 53_2
[IGNORED] Empty dir: 65_2
[IGNORED] Empty dir: 30_2
[IGNORED] Empty dir: 103_2
[IGNORED] Empty dir: 68_2
[IGNORED] Empty dir: 66_2
[IGNORED] Empty dir: 11_2
[IGNORED] Empty dir: 101_4
[IGNORED] Empty dir: 111_2
[IGNORED] Empty dir: 77_2
[IGNORED] Empty dir: 14_2
[IGNORED] Empty dir: 101_2
[IGNORED] Empty dir: 12_2
[IGNORED] Empty dir: 23_2
[IGNORED] Empty dir: 95_2
[IGNORED] Empty dir: 61_2
[IGNORED] Empty dir: 29_2
[IGNORED] Empty dir: 10_4
[IGNORED] Empty dir: 64_2
[IGNORED] Empty dir: 87_4
[IGNORED] Empty dir: 83_2
[IGNORED] Empty dir: 31_4
[IGNORED] Empty dir: 62_2
[IGNORED] Empty dir: 113_2
[IGNORED] Empty dir: 20_2
[IGNORED] Empty dir: 70_2
[IGNORED] Empty dir: 25_2
[IGNORE

In [88]:
running_number

625

# BELOW IS OLD (DECPRECATED)

# Save to .npy files

In [27]:
idle_data = np.load('saved_data/new_pushup_idle_class_len_8.npy')
idle_data.shape

(3220, 8, 99)

In [28]:
idle_data[0,:].shape

(8, 99)

In [29]:
# result number to the next cell
idle_data.shape[0] / target_vid

14.907407407407407

In [30]:
# result number to the next cell
idle_data.shape[0] / 15

214.66666666666666

In [31]:
arr_split = np.array_split(idle_data, 215)

In [32]:
len(arr_split), arr_split[0].shape

(215, (15, 8, 99))

## Split vid level saving
Just same amount of folder

In [33]:
# split vid level
running_idx = 0
r = 1
per_dir = 15
save_dir = Path('/home/u6188041/jom/Dataset/EzFit_dataset/keypoints_seq/len_8/pushup/idle')
for out_idx, arr in tqdm(enumerate(arr_split)):
    base_dir = save_dir / f'idle_{out_idx}'
    if not base_dir.exists():
        base_dir.mkdir(parents=True)
    for i in range(arr.shape[0]):
        save_name = base_dir / (stacked_names[running_idx] + f'_r{r}_{i}')
        selected = arr[i,:]
        np.save(save_name, selected)
        running_idx += 1
        if running_idx != 0 and running_idx % og_len == 0:
            r += 1

215it [00:00, 489.16it/s]


## Seq level saving

In [20]:
# seq level
save_dir = Path('D:/EzFit_dataset/keypoints_seq/len_8/half-burpee/idle')
for idx in tqdm(range(idle_data.shape[0])):
    base_dir = save_dir / str(idx)
    if not base_dir.exists():
        base_dir.mkdir(parents=True)
    selected = idle_data[idx,:]
    np.save(base_dir / f'idle_{idx}', selected)

100%|█████████████████████████████████████████████████████████████████████████████| 1422/1422 [00:03<00:00, 440.29it/s]
